In [47]:
import tensorflow as tf
import numpy as np
from tqdm import tqdm_notebook as tqdm

In [48]:
sentence = " if you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to log for the endless immensity of the sea"

dic_size = len(char_set)  # RNN input size (one hot size)
hidden_size = len(char_set)  # RNN output size
num_classes = len(char_set)  # final output size (RNN or softmax, etc.)
batch_size = 1  # one sample data, one batch
seq_length = 10  # number of lstm rollings (unit #)
learning_rate = 0.1

char_set = list(set(sentence))
char_dic = {w: i for i, w in enumerate(char_set)}

dataX = []
dataY = []

for i in range(0, len(sentence) - seq_length):
    x_str = sentence[i:i + seq_length]
    y_str = sentence[i + 1:i+ seq_length + 1]
    #print(i, x_str, '->', y_str)
    
    x = [char_dic[c] for c in x_str]
    y = [char_dic[c] for c in y_str]
    
    dataX.append(x)
    dataY.append(y)
    
batch_size = len(dataX)

In [55]:
X = tf.placeholder(tf.int32, [None, seq_length])
Y = tf.placeholder(tf.int32, [None, seq_length])

x_one_hot = tf.one_hot(X, num_classes)

#tf.reset_default_graph()

with tf.variable_scope("cell", reuse=None) as scope:
    cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
    cell = tf.contrib.rnn.MultiRNNCell([cell] * 2, state_is_tuple=True)
    initial_state = cell.zero_state(batch_size, tf.float32)
    outputs, _states = tf.nn.dynamic_rnn(
        cell, x_one_hot, initial_state=initial_state, dtype=tf.float32)
    
    print(outputs.shape)
    
    X_for_softmax = tf.reshape(outputs, [-1, hidden_size])
    
    softmax_w = tf.get_variable("softmax_w", 
                               [hidden_size, num_classes])
    softmax_b = tf.get_variable("softmax_b",
                               [num_classes])
    
    outputs = tf.matmul(X_for_softmax, softmax_w) + softmax_b
    
    outputs = tf.reshape(outputs, [batch_size, seq_length, num_classes])
    
    print(outputs.shape)
    
    weights = tf.ones([batch_size, seq_length])
    seq_loss = tf.contrib.seq2seq.sequence_loss(logits=outputs, targets=Y, weights=weights)
    
    loss = tf.reduce_mean(seq_loss)
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
    
    prediction = tf.argmax(outputs, axis=2)

(169, 10, 24)
(169, 10, 24)


In [57]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for i in tqdm(range(1000), leave=True):
        l, _ = sess.run([loss, optimizer], feed_dict={X: dataX, Y: dataY})
        resultset = sess.run(prediction, feed_dict={X: dataX})
        
        if i % 100 == 0:
            for j, result in enumerate(resultset):
                print(i, j, ''.join([char_set[t] for t in result]), l)
            
        result_str = [char_set[c] for c in np.squeeze(result)]
        
        if i % 1 == 0:
            print(i, "loss:", l, "\tPrediction:", ''.join(result_str))
            

0 0 rl         3.26436
0 1 rl         3.26436
0 2 rl         3.26436
0 3 rll        3.26436
0 4 rlll       3.26436
0 5 rlll       3.26436
0 6 rll        3.26436
0 7 rll        3.26436
0 8 rll        3.26436
0 9 rl         3.26436
0 10 rll        3.26436
0 11 rllll      3.26436
0 12 rll        3.26436
0 13 rlll       3.26436
0 14 rll        3.26436
0 15 rl         3.26436
0 16 rl         3.26436
0 17 rl         3.26436
0 18 rl         3.26436
0 19 rll        3.26436
0 20 rll        3.26436
0 21 rll        3.26436
0 22 rll        3.26436
0 23 rll        3.26436
0 24 rlll       3.26436
0 25 rll        3.26436
0 26 rl         3.26436
0 27 rlll       3.26436
0 28 rl         3.26436
0 29 rll        3.26436
0 30 rl         3.26436
0 31 rll        3.26436
0 32 rl         3.26436
0 33 rll        3.26436
0 34 rll        3.26436
0 35 rl         3.26436
0 36 rh         3.26436
0 37 rl         3.26436
0 38 rll        3.26436
0 39 rll        3.26436
0 40 rlll       3.26436
0 41 rllll      3.26436
0 

100 79 ton't assi 0.235181
100 80  n't assig 0.235181
100 81  't assign 0.235181
100 82 dt assign  0.235181
100 83 t assign t 0.235181
100 84  dssign th 0.235181
100 85 tnsign the 0.235181
100 86 nkign them 0.235181
100 87 iign them  0.235181
100 88 itn them t 0.235181
100 89 pn them ta 0.235181
100 90   them tas 0.235181
100 91 dthem task 0.235181
100 92 toem tasks 0.235181
100 93  em tasks  0.235181
100 94 em tasks a 0.235181
100 95 n tasks an 0.235181
100 96  tosks and 0.235181
100 97 tosks and  0.235181
100 98  sks and w 0.235181
100 99 nks and wo 0.235181
100 100 is and wor 0.235181
100 101 , and work 0.235181
100 102 iind work, 0.235181
100 103 tnd dork,  0.235181
100 104 nd dork, b 0.235181
100 105 d dork, bu 0.235181
100 106  aork, but 0.235181
100 107 took, but  0.235181
100 108 ork, but r 0.235181
100 109   , but ra 0.235181
100 110  , but rat 0.235181
100 111 , but rath 0.235181
100 112  dut rathe 0.235181
100 113 tui rather 0.235181
100 114 ut rather  0.235181
100 115 m rat

200 90   them tas 0.230052
200 91 dthem task 0.230052
200 92 toem tosks 0.230052
200 93  em tosks  0.230052
200 94 em tosks a 0.230052
200 95 n tosks an 0.230052
200 96  tosks and 0.230052
200 97 tosks and  0.230052
200 98  sks and w 0.230052
200 99 nks and wo 0.230052
200 100  s and wor 0.230052
200 101 , and work 0.230052
200 102  ind work, 0.230052
200 103 tnd dork,  0.230052
200 104 nd dork, b 0.230052
200 105 d work, bu 0.230052
200 106  aork, but 0.230052
200 107 took, but  0.230052
200 108 ook, but r 0.230052
200 109  k, but ra 0.230052
200 110  , but rat 0.230052
200 111 , but rath 0.230052
200 112  but rathe 0.230052
200 113 tut rather 0.230052
200 114 ui rather  0.230052
200 115 m rather t 0.230052
200 116  dather te 0.230052
200 117 tather tea 0.230052
200 118  ther teac 0.230052
200 119 nher teach 0.230052
200 120  em toach  0.230052
200 121 em toach t 0.230052
200 122 n toach th 0.230052
200 123  toach the 0.230052
200 124 toach them 0.230052
200 125  ach them  0.230052
20

300 80  n't dssig 0.229239
300 81  't dssign 0.229239
300 82 dt dssign  0.229239
300 83 t dssign t 0.229239
300 84 htssign th 0.229239
300 85 tnsign the 0.229239
300 86 nsign them 0.229239
300 87 iign them  0.229239
300 88 itn them t 0.229239
300 89 ln them ta 0.229239
300 90   them tas 0.229239
300 91 dthem task 0.229239
300 92 toem tosks 0.229239
300 93 hem tosks  0.229239
300 94 er tosks a 0.229239
300 95 n tasks an 0.229239
300 96  tasks and 0.229239
300 97 tosks and  0.229239
300 98 hsks and w 0.229239
300 99 nss and wo 0.229239
300 100 is and wor 0.229239
300 101 s and work 0.229239
300 102 iind work, 0.229239
300 103 tnd work,  0.229239
300 104 nd work, b 0.229239
300 105 d work, bu 0.229239
300 106  aork, but 0.229239
300 107 took, but  0.229239
300 108 ork, but r 0.229239
300 109  k, but ra 0.229239
300 110  , but rat 0.229239
300 111 s but rath 0.229239
300 112  dut rathe 0.229239
300 113 tui rather 0.229239
300 114 ui rather  0.229239
300 115 m rather t 0.229239
300 116 htat

405 loss: 0.228966 	Prediction: tf the sea
406 loss: 0.228947 	Prediction: tf the sea
407 loss: 0.228952 	Prediction: tf the sea
408 loss: 0.228957 	Prediction: tf the sea
409 loss: 0.228935 	Prediction: tf the sea
410 loss: 0.228935 	Prediction: tf the sea
411 loss: 0.228939 	Prediction: tf the sea
412 loss: 0.228931 	Prediction: tf the sea
413 loss: 0.228928 	Prediction: tf the sea
414 loss: 0.228919 	Prediction: tf the sea
415 loss: 0.228918 	Prediction: tf the sea
416 loss: 0.228922 	Prediction: tf the sea
417 loss: 0.228913 	Prediction: tf the sea
418 loss: 0.228911 	Prediction: tf the sea
419 loss: 0.228914 	Prediction: tf the sea
420 loss: 0.228913 	Prediction: tf the sea
421 loss: 0.228921 	Prediction: tf the sea
422 loss: 0.228942 	Prediction: tf the sea
423 loss: 0.228994 	Prediction: tf the sea
424 loss: 0.229137 	Prediction: tf the sea
425 loss: 0.229365 	Prediction: tf the sea
426 loss: 0.22959 	Prediction: tf the sea
427 loss: 0.22958 	Prediction: tf the sea
428 loss: 0.2

507 loss: 0.230343 	Prediction: tf the sea
508 loss: 0.229808 	Prediction: tf the sea
509 loss: 0.22938 	Prediction: tf the sea
510 loss: 0.22937 	Prediction: tf the sea
511 loss: 0.229366 	Prediction: tf the sea
512 loss: 0.229336 	Prediction: tf the sea
513 loss: 0.229307 	Prediction: tf the sea
514 loss: 0.229206 	Prediction: tf the sea
515 loss: 0.228999 	Prediction: tf the sea
516 loss: 0.229118 	Prediction: tf the sea
517 loss: 0.229137 	Prediction: tf the sea
518 loss: 0.228995 	Prediction: tf the sea
519 loss: 0.228995 	Prediction: tf the sea
520 loss: 0.228991 	Prediction: tf the sea
521 loss: 0.228965 	Prediction: tf the sea
522 loss: 0.228973 	Prediction: tf the sea
523 loss: 0.22892 	Prediction: tf the sea
524 loss: 0.228897 	Prediction: tf the sea
525 loss: 0.228913 	Prediction: tf the sea
526 loss: 0.228899 	Prediction: tf the sea
527 loss: 0.228868 	Prediction: tf the sea
528 loss: 0.228877 	Prediction: tf the sea
529 loss: 0.228873 	Prediction: tf the sea
530 loss: 0.22

603 loss: 0.229014 	Prediction: tf the sea
604 loss: 0.228925 	Prediction: tf the sea
605 loss: 0.228863 	Prediction: tf the sea
606 loss: 0.228865 	Prediction: tf the sea
607 loss: 0.228904 	Prediction: tf the sea
608 loss: 0.228958 	Prediction: tf the sea
609 loss: 0.22897 	Prediction: tf the sea
610 loss: 0.228908 	Prediction: tf the sea
611 loss: 0.228789 	Prediction: tf the sea
612 loss: 0.228726 	Prediction: tf the sea
613 loss: 0.228776 	Prediction: tf the sea
614 loss: 0.228846 	Prediction: tf the sea
615 loss: 0.228839 	Prediction: tf the sea
616 loss: 0.228788 	Prediction: tf the sea
617 loss: 0.228755 	Prediction: tf the sea
618 loss: 0.228748 	Prediction: tf the sea
619 loss: 0.228741 	Prediction: tf the sea
620 loss: 0.228739 	Prediction: tf the sea
621 loss: 0.228748 	Prediction: tf the sea
622 loss: 0.228748 	Prediction: tf the sea
623 loss: 0.228735 	Prediction: tf the sea
624 loss: 0.228724 	Prediction: tf the sea
625 loss: 0.228719 	Prediction: tf the sea
626 loss: 0.

707 loss: 0.228718 	Prediction: tf the sea
708 loss: 0.228752 	Prediction: tf the sea
709 loss: 0.228815 	Prediction: tf the sea
710 loss: 0.228908 	Prediction: tf the sea
711 loss: 0.229067 	Prediction: tf the sea
712 loss: 0.229216 	Prediction: tf the sea
713 loss: 0.229378 	Prediction: tf the sea
714 loss: 0.229349 	Prediction: tf the sea
715 loss: 0.229185 	Prediction: tf the sea
716 loss: 0.228913 	Prediction: tf the sea
717 loss: 0.228711 	Prediction: tf the sea
718 loss: 0.228681 	Prediction: tf the sea
719 loss: 0.228786 	Prediction: tf the sea
720 loss: 0.228893 	Prediction: tf the sea
721 loss: 0.228886 	Prediction: tf the sea
722 loss: 0.228776 	Prediction: tf the sea
723 loss: 0.228689 	Prediction: tf the sea
724 loss: 0.228699 	Prediction: tf the sea
725 loss: 0.228761 	Prediction: tf the sea
726 loss: 0.228778 	Prediction: tf the sea
727 loss: 0.228727 	Prediction: tf the sea
728 loss: 0.228681 	Prediction: tf the sea
729 loss: 0.228686 	Prediction: tf the sea
730 loss: 0

807 loss: 0.228634 	Prediction: tf the sea
808 loss: 0.228626 	Prediction: tf the sea
809 loss: 0.228627 	Prediction: tf the sea
810 loss: 0.228632 	Prediction: tf the sea
811 loss: 0.228633 	Prediction: tf the sea
812 loss: 0.228631 	Prediction: tf the sea
813 loss: 0.228626 	Prediction: tf the sea
814 loss: 0.228621 	Prediction: tf the sea
815 loss: 0.22862 	Prediction: tf the sea
816 loss: 0.228621 	Prediction: tf the sea
817 loss: 0.228624 	Prediction: tf the sea
818 loss: 0.228625 	Prediction: tf the sea
819 loss: 0.228624 	Prediction: tf the sea
820 loss: 0.228626 	Prediction: tf the sea
821 loss: 0.228634 	Prediction: tf the sea
822 loss: 0.228657 	Prediction: tf the sea
823 loss: 0.228701 	Prediction: tf the sea
824 loss: 0.228778 	Prediction: tf the sea
825 loss: 0.228846 	Prediction: tf the sea
826 loss: 0.228903 	Prediction: tf the sea
827 loss: 0.228944 	Prediction: tf the sea
828 loss: 0.229035 	Prediction: tf the sea
829 loss: 0.229193 	Prediction: tf the sea
830 loss: 0.

900 168 tf the sea 0.228969
900 loss: 0.228969 	Prediction: tf the sea
901 loss: 0.229159 	Prediction: tf the sea
902 loss: 0.229198 	Prediction: tf the sea
903 loss: 0.229118 	Prediction: tf the sea
904 loss: 0.228961 	Prediction: tf the sea
905 loss: 0.228915 	Prediction: tf the sea
906 loss: 0.229028 	Prediction: tf the sea
907 loss: 0.22886 	Prediction: tf the sea
908 loss: 0.22876 	Prediction: tf the sea
909 loss: 0.228871 	Prediction: tf the sea
910 loss: 0.228868 	Prediction: tf the sea
911 loss: 0.228746 	Prediction: tf the sea
912 loss: 0.228735 	Prediction: tf the sea
913 loss: 0.228771 	Prediction: tf the sea
914 loss: 0.228734 	Prediction: tf the sea
915 loss: 0.228691 	Prediction: tf the sea
916 loss: 0.228731 	Prediction: tf the sea
917 loss: 0.228728 	Prediction: tf the sea
918 loss: 0.228674 	Prediction: tf the sea
919 loss: 0.228675 	Prediction: tf the sea
920 loss: 0.228692 	Prediction: tf the sea
921 loss: 0.228659 	Prediction: tf the sea
922 loss: 0.228666 	Predicti